In [1]:
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import Node2Vec
torch_geometric.__version__
import gc
import polars as pl
import torch_cluster

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [2]:
train_df = pl.read_parquet('../data/train.parquet',
                           columns=['session','aid'],
                           low_memory= True,
                          )
test_df = pl.read_parquet('../data/test.parquet',columns=['session','aid'])

In [3]:
# # 이전 세션(session)의 aid 값을 포함하는 그래프 데이터 저장

# def lagged_df(df):
#     # 이전 세션(session)의 aid 값을 새로운 열(column)로 추가
#     df = df.with_column(pl.col("aid").shift(periods=1).over("session")
#                               #.cast(pl.Int32)
#                               #.fill_null(pl.col("aid"))
#                               .alias("prev_aid"))
#     return df
# train_df = lagged_df(train_df)
# test_df = lagged_df(test_df)

# df = pl.concat([
#     train_df,
#     test_df
# ], how="vertical")
# edges_torch_T = torch.tensor(np.transpose(df[['prev_aid','aid']].to_numpy()),dtype=torch.long)
# torch.save(edges_torch_T,"../output/all_edges_train_and_test.pt")

C:\Users\wikid\AppData\Local\Temp\ipykernel_17820\738683804.py:5: DeprecationWarning: `DataFrame.with_column` has been renamed; this redirect is temporary, please use `.with_columns` instead
  df = df.with_column(pl.col("aid").shift(periods=1).over("session")


이 코드는 이전 세션(session)의 aid 값을 포함하는 그래프 데이터를 저장하는 코드이다. 아래는 코드의 각 줄에 대한 설명이다.

def lagged_df(df):: df라는 데이터프레임(DataFrame)을 입력으로 받는 함수를 정의한다.

df = df.with_column(...): df 데이터프레임에 새로운 열(column)을 추가한다. pl.col("aid").shift(periods=1).over("session").alias("prev_aid")는 session 열을 기준으로 aid 열을 한 행씩 미루어서(shift) 이전 세션(session)의 aid 값을 가져와(over) prev_aid 열로 추가한다.

return df: 열이 추가된 데이터프레임을 반환한다.

train_df = lagged_df(train_df): train_df 데이터프레임에 이전 세션(session)의 aid 값을 포함하는 prev_aid 열을 추가한다.

test_df = lagged_df(test_df): test_df 데이터프레임에 이전 세션(session)의 aid 값을 포함하는 prev_aid 열을 추가한다.

df = pl.concat([...], how="vertical"): train_df와 test_df 데이터프레임을 수직(vertical)으로 합칩니다. how="vertical" 인자는 수직으로 합친다는 의미이다.

edges_torch_T = torch.tensor(..., dtype=torch.long): 합쳐진 데이터프레임에서 prev_aid와 aid 열을 뽑아서 NumPy 배열로 변환한 뒤, 이를 텐서(Tensor)로 변환한다. dtype=torch.long은 텐서의 자료형(data type)을 long으로 설정한다.

torch.save(edges_torch_T,"all_edges_train_and_test.pt"): 텐서 edges_torch_T를 파일 형식(.pt)으로 저장한다. 저장된 파일 이름은 all_edges_train_and_test.pt이다.

In [4]:
edges_tensor = torch.load("../output/all_edges_train_and_test.pt")

In [5]:
# 그래프 데이터 생성
data = Data(edge_index=edges_tensor)
# edges_tensor라는 텐서(tensor)를 이용하여 그래프 데이터를 생성
print(data)

Data(edge_index=[2, 223644219])


Data 클래스는 그래프 데이터를 저장하는 데 사용되는 클래스.
edge_index 속성은 각 엣지(edge)의 시작 노드와 끝 노드를 표시하는 텐서(tensor).
edge_index 텐서를 이용하여 그래프의 연결 상태를 나타내는 그래프 데이터를 생성할 수 있다.

In [ ]:
del edges_tensor
gc.collect()

In [ ]:
model = Node2Vec(data.edge_index, embedding_dim=32, 
                 walk_length=10,                        # lenght of rw
                 context_size=5, walks_per_node=10,
                 num_negative_samples=2, 
                 p=0.2, q=0.5,                             # bias parameters
                 sparse=True).to(device)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

data.edge_index: 그래프에서의 엣지 정보를 나타내는 텐서.    
embedding_dim=32: 노드 임베딩 벡터의 차원 수를 정의.    
walk_length=10: Random Walk의 길이를 정의.    
context_size=5: Skip-Gram 모델에서 사용되는 문맥 크기를 정의.    
walks_per_node=10: 노드마다 실행되는 Random Walk의 횟수를 정의.    
num_negative_samples=2: Skip-Gram 모델에서 사용되는 음수 샘플링의 수를 정의.    
p=0.2, q=0.5: Node2Vec에서 사용되는 Random Walk의 바이어스 파라미터. p는 이전 노드로 돌아가기 쉬운 경로를 선호하는 경향이 있으며, q는 더 멀리 이동하는 경로를 선호하는 경향이 있다.    
sparse=True: 엣지 인덱스가 희소 텐서일 경우 성능을 향상시키기 위해 True로 설정.    
.to(device): 모델을 CPU 또는 GPU와 같은 디바이스로 이동.    

In [ ]:
# 데이터 로더(loader)와 옵티마이저(optimizer) 설정

loader = model.loader(batch_size=128, shuffle=True,
                      num_workers=6)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

이 코드는 파이토치(PyTorch) 라이브러리를 사용하여 딥러닝 모델을 학습할 때 사용될 데이터 로더(loader)와 옵티마이저(optimizer)를 설정하는 부분이다.

데이터 로더 설정
데이터 로더(loader)는 학습 데이터를 불러오고 배치(batch) 단위로 분할하여 모델에 입력으로 제공하는 역할을 한다. 이 코드에서는 model이라는 객체에 대해 batch_size를 128, shuffle을 True로 하고, num_workers를 2로 설정하여 데이터 로더를 만들고 있다.

batch_size: 모델이 한 번에 처리할 데이터 개수
shuffle: 데이터를 섞을지 여부
num_workers: 데이터 로딩을 위해 사용되는 worker(process) 수
옵티마이저 설정
옵티마이저(optimizer)는 학습 중 모델의 가중치(weight)를 업데이트하는 알고리즘이다. 이 코드에서는 SparseAdam 옵티마이저를 사용하며, model의 파라미터를 입력으로 받아 학습률(learning rate)을 0.01로 설정하고 있다.

SparseAdam: Adam 알고리즘 중 희소(sparse)한 경우에 효과적인 버전
list(model.parameters()): 모델의 파라미터를 리스트로 변환하여 옵티마이저에 입력하기 위한 작업이다. 이 코드에서는 모델의 모든 파라미터를 학습 대상으로 설정한다.

In [ ]:
del data
gc.collect()

In [ ]:
 def train():
        model.train()
        total_loss = 0
        for pos_rw, neg_rw in tqdm(loader):
            optimizer.zero_grad()
            loss = model.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(loader)

for epoch in range(0, 12):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

model.train(): 모델을 학습 모드로 설정한다.

total_loss = 0: 전체 손실(loss)을 0으로 초기화한다.

for pos_rw, neg_rw in tqdm(loader):: 데이터 로더에서 배치(batch) 단위로 데이터를 불러온다. pos_rw는 긍정적인 예시(positive example)에 대한 정보이며, neg_rw는 부정적인 예시(negative example)에 대한 정보이다. tqdm은 반복문의 진행 상황을 보여주는 라이브러리이다.

optimizer.zero_grad(): 옵티마이저의 그래디언트(gradient)를 0으로 초기화한다.

loss = model.loss(pos_rw.to(device), neg_rw.to(device)): 모델에 입력으로 주어진 배치 데이터에 대해 손실을 계산한다. to(device)는 모델과 데이터를 GPU로 옮기기 위한 작업이다.

loss.backward(): 손실에 대한 그래디언트를 계산한다.

optimizer.step(): 옵티마이저를 사용하여 모델의 가중치(weight)를 업데이트한다.

total_loss += loss.item(): 전체 손실을 현재 배치의 손실로 업데이트한다.

return total_loss / len(loader): 모든 배치에 대한 학습이 끝나면, 전체 데이터에 대한 평균 손실을 반환한다.

for epoch in range(0, 12):: 전체 데이터에 대해 12번의 에포크(epoch)동안 학습을 수행한다.

loss = train(): train 함수를 호출하여 현재 에포크의 평균 손실을 계산한다.

print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}'): 현재 에포크와 평균 손실을 출력한다. f-string을 사용하여 문자열 내에서 변수를 포맷팅한다. :02d는 두 자리 숫자로 표현하며, :.4f는 소수점 이하 4자리까지 표현한다.

In [ ]:
%%time
from annoy import AnnoyIndex

index = AnnoyIndex(32, 'angular')

for idx,idx_embedding in enumerate(model.state_dict()['embedding.weight'].cpu()):
    index.add_item(idx, idx_embedding)
    
index.build(10)

이 코드는 Annoy 라이브러리를 사용하여 임베딩 벡터(embedding vector)를 검색하는 인덱스(index)를 생성하는 코드이다. 아래는 코드의 각 줄에 대한 설명이다.

from annoy import AnnoyIndex: Annoy 라이브러리에서 AnnoyIndex 클래스를 불러온다.

index = AnnoyIndex(32, 'angular'): 32차원의 임베딩 벡터를 가지는 Annoy 인덱스를 생성한다. 'angular'은 거리 측정 방식으로 유클리드 거리(euclidean distance) 대신 코사인 거리(cosine distance)를 사용하겠다는 의미이다.

for idx,idx_embedding in enumerate(model.state_dict()['embedding.weight'].cpu()):: 모델의 임베딩 레이어(embedding layer)의 가중치(weight)에 대해 반복문을 수행한다. enumerate 함수를 사용하여 인덱스와 값 쌍을 불러온다. cpu()는 모델의 가중치를 CPU 메모리로 옮기는 작업이다.

index.add_item(idx, idx_embedding): Annoy 인덱스에 임베딩 벡터를 추가한다. idx는 벡터의 인덱스(index)를, idx_embedding은 벡터 자체를 나타낸다.

index.build(10): Annoy 인덱스를 빌드(build)한다. 10은 인덱스 내에서 검색할 이웃(neighbors)의 개수이다. 이후 Annoy 인덱스는 주어진 쿼리(query)에 대해 가장 가까운 이웃 벡터를 반환할 수 있게 된다.

In [ ]:
embeddings_node2vec = model.cpu().state_dict()['embedding.weight'].numpy()

np.save("node2vec_embeddings",embeddings_node2vec)

이 코드는 학습된 모델의 임베딩 벡터(embedding vector)를 NumPy 배열로 변환하여 node2vec_embeddings.npy 파일로 저장하는 코드이다. 아래는 코드의 각 줄에 대한 설명이다.

embeddings_node2vec = model.cpu().state_dict()['embedding.weight'].numpy(): 모델의 임베딩 레이어(embedding layer)의 가중치(weight)를 불러온다. cpu()는 모델의 가중치를 CPU 메모리로 옮기는 작업이다. numpy()는 PyTorch 텐서(Tensor)를 NumPy 배열로 변환한다.

np.save("node2vec_embeddings", embeddings_node2vec): embeddings_node2vec 배열을 NumPy 배열 파일 형식(.npy)으로 저장한다. 저장된 파일 이름은 node2vec_embeddings.npy 이다.

In [ ]:
del model, loader, optimizer, embeddings_node2vec
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Validation / Inference

In [ ]:
def evaluate(path,mode="validation",n_neighbors=12):


    test = pl.read_parquet(path)

    session_types = ['clicks', 'carts', 'orders']
    test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
    test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)

    del test
    gc.collect()
    labels = []

    type_weight_multipliers = {0: 1, 1: 6, 2: 3}

    for AIDs, types in zip(test_session_AIDs, test_session_types):
        if len(AIDs) >= 20:
                # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
            weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
            aids_temp=defaultdict(lambda: 0)
            for aid,w,t in zip(AIDs,weights,types): 
                aids_temp[aid]+= w * type_weight_multipliers[t]

            sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
            labels.append(sorted_aids[:20])
        else:
            # here we don't have 20 aids to output -- we will use word2vec embeddings to generate candidates!
            AIDs = list(dict.fromkeys(AIDs[::-1]))

            # let's grab the most recent aid
            most_recent_aid = AIDs[0]

            # and look for some neighbors!
            nns = [i for i in index.get_nns_by_item(most_recent_aid, n_neighbors+1)[1:]]


            labels.append((AIDs+nns)[:n_neighbors])

    labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

    predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

    prediction_dfs = []

    for st in session_types:
        modified_predictions = predictions.copy()
        modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
        prediction_dfs.append(modified_predictions)

    sub = pd.concat(prediction_dfs).reset_index(drop=True)
    
    del prediction_dfs, predictions,labels_as_strings, labels, test_session_types,test_session_AIDs
    gc.collect()
    if mode=="test":
        sub.to_csv("submission.csv",index=False)
        return sub
    else:

        sub['labels_2'] = sub['labels'].apply(lambda x : [int(s) for s in x.split(' ')])
        submission = pd.DataFrame()
        submission['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
        submission['type'] = sub.session_type.apply(lambda x: x.split('_')[1])
        submission['labels'] = sub.labels_2.apply(lambda x : [item for item in x[:] ]) #.apply(lambda x: [int(i) for i in x.split(',')[:20]])
        test_labels = pd.read_parquet('/kaggle/input/otto-train-and-test-data-for-local-validation/test_labels.parquet')
        test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
        del sub,submission
        gc.collect()
        gc.collect()
        test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
        test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
        recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 
        score = (recall_per_type * pd.Series({'clicks': 0.1, 'carts': 0.30, 'orders': 0.60})).sum()

        return score

이 코드는 세션 기반 추천 시스템의 성능을 평가하기 위한 함수이다. evaluate 함수는 파일 경로를 입력으로 받으며, 해당 파일은 test 데이터 또는 validation 데이터이다. 함수의 mode 인수를 사용하여 데이터의 유형을 설정할 수 있다.

해당 함수는 먼저 입력된 데이터를 읽어와서, 각 세션에서 사용된 모든 상품 ID(aid)와 세션 유형(type)을 추출한다. 이후 각 세션에 대해 다음을 수행한다.

세션에 포함된 aid의 수가 20개 이상인 경우, 가장 최근의 aid를 제외한 나머지 aid들을 기반으로 가중치를 계산한다. 가중치는 각 aid가 발생한 시점에서 가중치가 더 크게 부여된다. 마지막으로 가중치가 가장 높은 상위 20개 aid를 레이블로 지정한다.

세션에 포함된 aid의 수가 20개 미만인 경우, 해당 세션에 포함된 aid와 최근 aid의 이웃을 찾아서, 그 이웃들을 레이블로 지정한다. 레이블의 수는 n_neighbors 인수로 지정된다.

이후, 각 세션의 레이블을 문자열 형태로 변환하고, 이를 데이터프레임으로 저장한다. 마지막으로 저장된 데이터프레임을 기반으로 성능 지표를 계산하여 반환한다. 성능 지표는 각 세션 유형마다 다른 가중치를 부여한 후, 그 가중치의 합으로 계산된다.

In [ ]:
path = "/kaggle/input/otto-train-and-test-data-for-local-validation/test.parquet"
validation_score = evaluate(path,mode="validation",n_neighbors=20)

In [ ]:
path = "/kaggle/input/otto-full-optimized-memory-footprint/test.parquet"
test_submission = evaluate(path,mode="test",n_neighbors=20)

Node2Vec 모델의 성능을 향상하기 위해 다음과 같은 시도를 할 수 있다:

하이퍼파라미터 튜닝: Node2Vec 모델은 여러 하이퍼파라미터를 가지고 있으며, 이들은 모델의 성능에 영향을 미칩니다. 따라서, 하이퍼파라미터를 조정하여 최적의 조합을 찾아내는 것이 필요한다.

더 많은 데이터 수집: 더 많은 데이터를 수집하여 모델의 성능을 향상시킬 수 있다. 이는 모델이 더 많은 패턴을 학습할 수 있도록 도와줍니다.

다른 임베딩 방법 사용: Node2Vec는 그래프 임베딩 방법 중 하나이다. 다른 임베딩 방법을 사용해 볼 수도 있다. 예를 들어, DeepWalk, LINE, GraphSAGE 등의 방법이 있다.

다른 모델과 앙상블: Node2Vec 모델과 다른 그래프 임베딩 모델을 앙상블하여 최종 예측을 수행할 수도 있다.

그래프 구조 개선: Node2Vec 모델의 성능은 그래프의 구조에 크게 영향을 받습니다. 따라서, 그래프 구조를 개선하는 것도 모델의 성능을 향상시키는 데에 도움이 될 수 있다. 예를 들어, 그래프의 노드 간 연결성을 더욱 강화하거나, 더 많은 노드와 연결되도록 그래프를 확장하는 등의 방법이 있다.

하이퍼파라미터 튜닝을 위해서는 먼저 어떤 하이퍼파라미터가 있는지 이해해야 한다. Node2Vec 모델의 경우, 다음과 같은 하이퍼파라미터가 있을 수 있다.

p: random walk에서 이전 노드로 돌아가기 위한 확률
q: random walk에서 다음 노드로 진행하기 위한 확률
num_walks: 노드 당 random walk 수
walk_length: random walk의 길이
dimensions: 임베딩 차원 수
window_size: skip-gram 모델에서 사용되는 윈도우 크기
workers: 학습에 사용되는 스레드 수
하이퍼파라미터 튜닝을 위해 다음과 같은 방법을 사용할 수 있다.

그리드 탐색(Grid Search): 모든 가능한 하이퍼파라미터 조합을 시도하여 최상의 조합을 찾는 방법이다. 이 방법은 모든 조합을 시도하기 때문에 계산 비용이 높을 수 있다.

랜덤 탐색(Random Search): 하이퍼파라미터 공간에서 랜덤한 조합을 선택하여 시도하는 방법이다. 이 방법은 계산 비용이 낮지만 최상의 조합을 찾는데 시간이 더 오래 걸릴 수 있다.

베이지안 최적화(Bayesian Optimization): 이전 반복에서 성능이 좋은 하이퍼파라미터 조합을 사용하여 다음 반복에서 더 나은 조합을 찾는 방법이다. 이 방법은 그리드 탐색보다 계산 비용이 낮으면서 최적의 조합을 더 빠르게 찾을 수 있다.

하이퍼파라미터 튜닝을 위해 가장 적합한 방법은 문제와 데이터에 따라 다르므로 여러 방법을 시도해보고 결과를 비교하여 가장 좋은 성능을 내는 방법을 선택해야 한다.







베이지안 최적화를 적용하기 위해서는 bayesian-optimization 라이브러리가 필요한다.

In [ ]:
!pip install bayesian-optimization
!pip install --upgrade colorama

In [1]:
from bayes_opt import BayesianOptimization
from torch_geometric.nn import Node2Vec


# 하이퍼파라미터 최적화 대상 함수 정의
def evaluate_embeddings(dimension, walk_length, num_walks, window_size, workers, p, q):
    # node2vec 모델 학습 및 평가
    # 학습 및 평가 코드 작성
    # return 값은 최적화하고자 하는 평가 지표

    # 모델 학습
    node2vec = Node2Vec(graph, dimensions=dimension, walk_length=walk_length, num_walks=num_walks, workers=workers, p=p, q=q)
    model = node2vec.fit(window=window_size)

    # 임베딩 추출
    embeddings = {}
    for node in graph.nodes():
        embeddings[node] = model.wv[node]

    # 분류 성능 평가
    X_train, X_test, y_train, y_test = train_test_split(embeddings.values(), labels, test_size=0.3, random_state=42)
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')

    return f1   # 대회 평가지표로 수정 필요##########


# 하이퍼파라미터 범위 지정
pbounds = {'dimension': (16, 128),
           'walk_length': (10, 80),
           'num_walks': (10, 40),
           'window_size': (2, 10),
           'workers': (1, 8),
           'p': (0.1, 1.0),
           'q': (0.1, 1.0)}

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(
    f=evaluate_embeddings,
    pbounds=pbounds,
    random_state=42,
)

# 하이퍼파라미터 최적화 수행
optimizer.maximize(n_iter=10)

# 최적화 결과 출력
print(optimizer.max)


evaluate_embeddings 함수에서는 node2vec 모델 학습과 평가를 수행하고, 최적화하고자 하는 평가 지표를 반환한다. 이 함수 내에서 하이퍼파라미터를 설정한 후, node2vec 모델을 학습하고 평가한다. 최적화하고자 하는 평가 지표는 이전에 언급한 바와 같이 F1-score나 AUC 등이 될 수 있다.

pbounds 변수에는 최적화할 하이퍼파라미터의 범위를 지정한다. 이 예시에서는 dimension, walk_length, num_walks, window_size, workers, p, q를 최적화 대상으로 설정했습니다.

BayesianOptimization 객체를 생성할 때는 f 인자에 evaluate_embeddings 함수를 지정하고, pbounds 인자에 pbounds 변수를 지정한다. 이후 maximize 메소드를 호출하여 최적화를 수행한다. n_iter 인자는 최적화할 횟수를 지정한다.

최적화 결과는 optimizer.max 속성에 저장되며, 최적의 하이퍼파라미터 값과 해당 값일 때의 평가 지표가 출력된다. 이 값을 기반으로 하이퍼파라미터를 조정하고 node2vec 모델을 재학습하여 성능을 향상시킬 수 있다.

In [ ]:
def evaluate_embeddings(dimensions, walk_length, num_walks, window_size, workers, p, q):
    """
    주어진 하이퍼파라미터를 사용하여 node2vec 모델을 학습하고, 임베딩을 추출하여
    분류 성능을 측정한다.

    :param dimensions: 임베딩 차원
    :param walk_length: random walk 길이
    :param num_walks: random walk 수
    :param window_size: 윈도우 크기
    :param workers: 병렬 처리 수
    :param p: return parameter
    :param q: in-out parameter
    :return: 분류 성능
    """
    # node2vec 모델 학습에 필요한 데이터 준비
    data = load_data()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    x = torch.tensor(data.x, dtype=torch.float32).to(device)
    y = torch.tensor(data.y, dtype=torch.long).to(device)
    edge_index = torch.tensor(data.edge_index, dtype=torch.long).to(device)
    edge_weight = torch.tensor(data.edge_weight, dtype=torch.float32).to(device)

    # node2vec 모델 학습
    node2vec = Node2Vec(edge_index=edge_index,
                        embedding_dim=dimensions,
                        walk_length=walk_length,
                        context_size=window_size,
                        walks_per_node=num_walks,
                        p=p, q=q,
                        sparse=True).to(device)

    model = node2vec.train(window_size=window_size)

    # 임베딩 추출
    embeddings = model.embedding.weight.cpu().detach().numpy()

    # 분류 성능 측정
    return train_and_evaluate(embeddings, x, y, workers=workers)
